In [11]:
%reload_ext autoreload
%autoreload 2
import torch
import os 

import pandas as pd
import numpy as np
# import scanpy as sc
import os
# from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

import functions as fx
import models as mx
import run as rx

import multiprocessing

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(multiprocessing.cpu_count())
torch.set_num_interop_threads(multiprocessing.cpu_count()) 

In [ ]:
ori_data = pd.read_csv("../data/processed/mitsialis_bengsch_rahman.csv", index_col="id")
ori_data = ori_data.sample(frac=1)
data = ori_data.sample(100)
data.columns

excluded_markers = ['pt', "source"]
data_markers = [col for col in data.columns if col not in excluded_markers]

scaler = MinMaxScaler()
data[data_markers] = scaler.fit_transform(data[data_markers])
data

data = data[~data.index.duplicated(keep='first')]
data.shape

In [4]:
dataset = fx.MyDataset(data[data_markers])


In [5]:
net = mx.VAE(x_dim=dataset.x_train.shape[1], device=device)
# net = MyDataParallel(net, device_ids=[0, 1])
net.to(device)

VAE(
  (fc1): Linear(in_features=10, out_features=50, bias=True)
  (fc21): Linear(in_features=50, out_features=5, bias=True)
  (fc22): Linear(in_features=50, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

In [10]:
import time
train_loader = DataLoader(dataset,shuffle=False,num_workers=0,batch_size=1000)
since = time.time()
base = time.time() - since
rx.train(net, learning_rate = .0001, epochs = 1,
      train_loader = train_loader, device=device)
cost = time.time() - since
print("batchsize 10",cost,cost/base)

Training loss for epoch 0 is 0.18391401, Reconstruction is 0.18070787, mmd is 0.00320613
batchsize 10 0.01548004150390625 310.66028708133973
